# Nuevas features

In [1]:
import pandas as pd

In [4]:
dataset_scl = pd.read_csv("../input_data/dataset_SCL.csv", parse_dates=True)
# Fecha-I
dataset_scl['Fecha-I'] = pd.to_datetime(dataset_scl['Fecha-I'])
dataset_scl['Fecha-O'] = pd.to_datetime(dataset_scl['Fecha-O'])
# Limpieza códigos
dataset_scl = dataset_scl[~pd.to_numeric(dataset_scl['Vlo-I'], errors='coerce').isnull()]
dataset_scl = dataset_scl[~pd.to_numeric(dataset_scl['Vlo-O'], errors='coerce').isnull()]

C:\Users\alexa\AppData\Local\Temp\ipykernel_16188\2238979313.py:1: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_scl = pd.read_csv("../input_data/dataset_SCL.csv", parse_dates=True)


**Temporada_alta**: 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y
30-Sep, 0 si no.

In [17]:
import datetime
datetime.datetime(2017, 3, 3)

datetime.datetime(2017, 3, 3, 0, 0)

In [18]:
import datetime

def temporada_alta(fecha):
    if fecha <= datetime.datetime(2017, 3, 3):
        return 1
    elif fecha >= datetime.datetime(2017, 12, 15):
        return 1
    elif datetime.datetime(2017, 7, 15) <= fecha <= datetime.datetime(2017, 7, 31):
        return 1
    elif datetime.datetime(2017, 9, 11) <= fecha <= datetime.datetime(2017, 9, 30):
        return 1
    else:
        return 0

In [19]:
dataset_scl['temporada_alta'] = dataset_scl['Fecha-I'].map(lambda x: temporada_alta(x))

**dif_min**: diferencia en minutos entre Fecha-O y Fecha-I.

In [55]:
dataset_scl['dif_min'] = (dataset_scl['Fecha-O'] - dataset_scl['Fecha-I']).dt.total_seconds()/60

**atraso_15**: 1 si dif_min > 15, 0 si no.

In [58]:
import numpy as np

dataset_scl['atraso_15'] = np.where(dataset_scl['dif_min'] > 15, 1, 0)

**periodo_dia**: mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche
(entre 19:00 y 4:59), en base a Fecha-I 

In [73]:
def periodo_dia(x):
    if x.time() < datetime.time(5):
        return 'noche'
    elif x.time() < datetime.time(12):
        return 'mañana'
    elif x.time() < datetime.time(19):
        return 'tarde'
    else:
        return 'noche'

In [74]:
dataset_scl['periodo_dia'] = dataset_scl['Fecha-I'].map(lambda x: periodo_dia(x))

In [75]:
dataset_scl.loc[:,['Fecha-I', 'periodo_dia']].sample(n=10)

,Fecha-I,periodo_dia
9301,2017-02-01 08:50:00,mañana
48248,2017-09-14 08:25:00,mañana
59734,2017-11-28 19:00:00,noche
15426,2017-03-22 19:00:00,noche
44836,2017-09-06 15:30:00,tarde
65998,2017-12-15 17:02:00,tarde
17516,2017-04-09 16:16:00,tarde
33419,2017-07-11 18:30:00,tarde
8452,2017-02-01 06:10:00,mañana
29051,2017-06-09 14:10:00,tarde


In [76]:
dataset_scl.to_csv('../clean_data/synthetic_features.csv', index=False)

In [83]:
synthetic = pd.read_csv('../clean_data/synthetic_features.csv', parse_dates=['Fecha-I', 'Fecha-O'])
print(synthetic.dtypes)
synthetic

Fecha-I           datetime64[ns]
Vlo-I                      int64
Ori-I                     object
Des-I                     object
Emp-I                     object
Fecha-O           datetime64[ns]
Vlo-O                    float64
Ori-O                     object
Des-O                     object
Emp-O                     object
DIA                        int64
MES                        int64
AÑO                        int64
DIANOM                    object
TIPOVUELO                 object
OPERA                     object
SIGLAORI                  object
SIGLADES                  object
temporada_alta             int64
dif_min                  float64
atraso_15                  int64
periodo_dia               object
dtype: object


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,temporada_alta,dif_min,atraso_15,periodo_dia
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226.0,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,1,3.0,0,noche
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226.0,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,1,9.0,0,noche
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226.0,SCEL,KMIA,AAL,...,2017,Martes,I,American Airlines,Santiago,Miami,1,9.0,0,noche
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226.0,SCEL,KMIA,AAL,...,2017,Miercoles,I,American Airlines,Santiago,Miami,1,3.0,0,noche
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226.0,SCEL,KMIA,AAL,...,2017,Jueves,I,American Airlines,Santiago,Miami,1,-2.0,0,noche
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68180,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,...,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,46.0,1,tarde
68181,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,...,2017,Lunes,I,JetSmart SPA,Santiago,Lima,1,16.0,1,tarde
68182,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,...,2017,Miercoles,I,JetSmart SPA,Santiago,Lima,1,40.0,1,tarde
68183,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,...,2017,Viernes,I,JetSmart SPA,Santiago,Lima,1,13.0,0,tarde
